[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/neuralangelo-colab/blob/main/colmap_colab.ipynb)

In [ ]:
%cd /content
# !apt -y install -qq aria2

!apt-get install \
    cmake \
    libgoogle-glog-dev \
    libgflags-dev \
    libatlas-base-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libfreeimage-dev \
    libmetis-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev

!wget https://huggingface.co/camenduru/neuralangelo/resolve/main/colmap.zip
!unzip /content/colmap.zip -d colmap

!cp -r /content/colmap/lib/. /usr/local/lib
!chmod 755 /content/colmap/bin/colmap
!cp -r /content/colmap/bin/. /usr/local/bin

!pip install -q commentjson addict pynvml wandb
!pip install -q https://huggingface.co/camenduru/CoDeF/resolve/main/tinycudann-1.7-cp310-cp310-linux_x86_64.whl

# %cd /content
# !git clone --recursive https://github.com/nvlabs/tiny-cuda-nn
# %cd /content/tiny-cuda-nn
# !cmake . -B build
# !cmake --build build --config RelWithDebInfo -j

%cd /content
!git clone -b dev --recursive https://github.com/camenduru/neuralangelo

# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/neuralangelo/resolve/main/toy_example.MOV -d /content/test -o toy_example.MOV
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/neuralangelo/resolve/main/neus.zip -d /content/datasets/dtu -o neus.zip
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/neuralangelo/resolve/main/trainingdata.zip -d /content/test -o trainingdata.zip
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/neuralangelo/raw/main/downloader.py -d /content -o downloader.py
# %cd /content/datasets/dtu
# !unzip -q neus.zip -d /content/datasets/dtu
# %cd /content/neuralangelo
# !python /content/neuralangelo/projects/neuralangelo/scripts/convert_dtu_to_json.py --dtu_path /content/datasets/dtu
# !bash projects/neuralangelo/scripts/preprocess_dtu.sh /content/datasets/dtu
# !python downloader.py --modality video --group both
# !python downloader.py --modality image --group intermediate
# !python downloader.py -s

In [ ]:
# !wget https://huggingface.co/camenduru/neuralangelo/resolve/main/fox.zip -O /content/fox.zip
# !unzip /content/fox.zip -d /content

# !mkdir /content/fox/sparse /content/fox/dense

# !colmap feature_extractor \
#     --database_path /content/fox/database.db \
#     --image_path /content/fox \
#     --ImageReader.camera_model=RADIAL \
#     --SiftExtraction.use_gpu=true \
#     --SiftExtraction.num_threads=32 \
#     --ImageReader.single_camera=true # assuming single camera

# !colmap sequential_matcher \
#     --database_path /content/fox/database.db \
#     --SiftMatching.use_gpu=true

# !colmap mapper \
#     --database_path /content/fox/database.db \
#     --image_path /content/fox \
#     --output_path /content/fox/sparse

# !colmap image_undistorter \
#     --image_path /content/fox \
#     --input_path /content/fox/sparse/0 \
#     --output_path /content/fox/dense \
#     --output_type COLMAP \
#     --max_image_size 2000

%cd /content/neuralangelo
!wget https://huggingface.co/camenduru/neuralangelo/resolve/main/lego.mp4 -O /content/lego.mp4
EXPERIMENT="lego"
PATH_TO_VIDEO="/content/lego.mp4"
SKIP_FRAME_RATE=2
SCENE_TYPE="object"
!bash /content/neuralangelo/projects/neuralangelo/scripts/preprocess.sh {EXPERIMENT} {PATH_TO_VIDEO} {SKIP_FRAME_RATE} {SCENE_TYPE}

In [ ]:
!wandb login

In [ ]:
%cd /content/neuralangelo
EXPERIMENT="lego"
GROUP="lego_group"
NAME="lego"
CONFIG=f"/content/neuralangelo/projects/neuralangelo/configs/custom/{EXPERIMENT}.yaml"
GPUS=1
CHECKPOINT_PATH="/content/checkpoint"
!torchrun --nproc_per_node={GPUS} train.py \
    --wandb \
    --wandb_name=neuralangelo \
    --logdir=logs/{GROUP}/{NAME} \
    --config={CONFIG} \
    --show_pbar

In [ ]:
# !rm -rf /content/neuralangelo/projects/neuralangelo/configs/custom/lego.yaml
# !rm -rf /content/neuralangelo/datasets/lego_skip2
# !rm -rf /content/neuralangelo/logs/lego_group